In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Import statistical module
from scipy import stats

# Import custom modules
import acquire
import prepare
import explore

# Import visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

## Wrangle

In [2]:
# Acquire and prepare data returned as dataframe
df = acquire.get_data(query_url = False)
print(df.shape)
df = prepare.prepare_dataframe(df)
train, validate, test = prepare.split_data(df)
train.shape, validate.shape, test.shape

Found file in working directory!
(70932, 83)


((53199, 63), (26599, 63), (26600, 63))

In [3]:
train.head()

,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,oversized,promo,reprint,variation,set_id,set,set_name,set_type,collector_number,digital,rarity,flavor_text,artist,border_color,frame,full_art,textless,booster,story_spotlight,edhrec_rank,penny_rank,prices,promo_types,security_stamp,produced_mana,watermark,frame_effects,loyalty,printed_name,color_indicator,life_modifier,hand_modifier,content_warning,flavor_name,usd,usd_foil,eur,eur_foil,is_foil,is_etched,is_glossy
12661,2d911053-a026-4b20-ba2d-dbcc367c1413,Ajani Goldmane,en,2010-07-16,normal,True,highres_scan,{2}{W}{W},4.0,Legendary Planeswalker — Ajani,+1: You gain 2 life.\n−1: Put a +1/+1 counter ...,0,0,[W],[W],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,True,False,485d2468-18c8-42a4-9482-ca1c51e0470e,m11,Magic 2011,core,1,False,mythic,no_flavor_text,Aleksi Briclot,black,2003,False,False,True,False,6746.0,1920.0,"{'usd': '8.02', 'usd_foil': '15.11', 'usd_etch...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,4,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,8.02,15.11,3.50,9.48,True,False,False
61300,dcb2e0d6-3181-4d3d-a3b4-3896288b2e0e,Champion of Lambholt,en,2020-08-07,normal,True,highres_scan,{1}{G}{G},3.0,Creature — Human Warrior,Creatures with power less than Champion of Lam...,1,1,[G],[G],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,True,False,372dafe8-b5d1-4b81-998f-3ae96b59498a,2xm,Double Masters,masters,156,False,rare,no_flavor_text,Christopher Moeller,black,2015,False,False,True,False,446.0,5238.0,"{'usd': '0.36', 'usd_foil': '3.22', 'usd_etche...",no_promo,oval,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.36,3.22,0.34,1.65,True,False,False
48604,af3aa149-00fa-4932-b004-f64c8b5d3ca7,Gargoyle,en,2009-07-17,token,True,highres_scan,,0.0,Token Artifact Creature — Gargoyle,Flying,3,4,[],[],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],False,False,False,False,6d9f49c1-4784-4fd6-a9f1-05e36e25befe,tm10,Magic 2010 Tokens,token,8,False,common,no_flavor_text,Paul Bonner,black,2003,False,False,True,False,0.0,0.0,"{'usd': '0.25', 'usd_foil': None, 'usd_etched'...",[setpromo],no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.25,0,0,0,False,False,False
55734,c8d75604-4941-4faf-8f1a-7fc529e64962,Goblin Settler,en,2022-04-12,normal,True,highres_scan,{3}{R},4.0,Creature — Goblin,"When Goblin Settler enters the battlefield, de...",1,1,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,True,True,"[nonfoil, foil]",False,False,True,False,4d92a8a7-ccb0-437d-abdc-9d70fc5ed672,sld,Secret Lair Drop,box,1076,False,rare,The safer outcroppings were already claimed by...,Roman Klonek,borderless,2015,False,False,False,False,14301.0,2011.0,"{'usd': '9.76', 'usd_foil': '11.87', 'usd_etch...",no_promo,oval,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,9.76,11.87,0,0,True,False,False
70638,feef03a1-f19a-413b-ad86-0b357cf9c09c,Weight Advantage,en,2016-08-26,normal,True,highres_scan,,0.0,Conspiracy,(Start the game with this conspiracy face up i...,0,0,[],[],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,True,True,"[nonfoil, foil]",False,False,False,False,ad1b8847-1905-4080-9e26-80691ea7c1ef,cn2,Conspiracy: Take the Crown,draft_innovation,12,False,rare,When immovable object turns to unstoppable for...,Jakub Kasper,black,2015,False,False,True,False,0.0,0.0,"{'usd': '0.11', 'usd_foil': '0.42', 'usd_etche...",no_promo,oval,no_produced_mana,conspiracy,[draft],0,no_printed_name,no_color_indic

In [4]:
def add_columns_with_usd_prices_for_styles(df):
    '''
    Adds columns to dataframe with the usd prices of the card style related fields.
    '''
    # Creates a column with USD prices for cards with frame effects
    df.loc[df['frame_effects'] != 'no_frame_effects', 'frame_effects_usd'] = df['usd']
    
    # Creates a column with USD prices for cards that don't have black borders versions
    df.loc[df['border_color'] != 'black', 'border_color_usd'] = df['usd']
    
    # Creates a column with USD prices cards without flavor text
    df.loc[df['flavor_text'] != 'no_flavor_text', 'flavor_text_usd'] = df['usd']
    
    # Creates a column with USD prices non-modern style frame cards
    df.loc[df['frame'] != '2015', 'frame_usd'] = df['usd']
    
    # Creates a column with USD prices for full art cards
    df.loc[df['full_art'] == True, 'full_art_usd'] = df['usd']
    
    # Creates a column with USD prices for highres_image cards
    df.loc[df['highres_image'] == True, 'highres_images_usd'] = df['usd']
    
    # Creates a column with USD prices for non-english card
    df.loc[df['lang'] != 'en', 'lang_usd'] = df['usd']
    
    # Creates a column with USD prices for cards with non-normal layouts
    df.loc[df['layout'] != 'normal', 'layout_usd'] = df['usd']
    
    # Creates a column with USD prices for promo cards
    df.loc[df['promo_types'] != 'no_promo', 'promo_types_usd'] = df['usd']
    
    # Creates a column with USD prices for cards without a security stamp
    df.loc[df['security_stamp'] != 'no_security_stamp', 'security_stamp_usd'] = df['usd']
    
    # Creates a column with USD prices for cards without a watermark
    df.loc[df['watermark'] != 'no_waterwark', 'watermark_usd'] = df['usd']
    return df

In [5]:
add_columns_with_usd_prices_for_styles(train)
train.head(3)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,oversized,promo,reprint,variation,set_id,set,set_name,set_type,collector_number,digital,rarity,flavor_text,artist,border_color,frame,full_art,textless,booster,story_spotlight,edhrec_rank,penny_rank,prices,promo_types,security_stamp,produced_mana,watermark,frame_effects,loyalty,printed_name,color_indicator,life_modifier,hand_modifier,content_warning,flavor_name,usd,usd_foil,eur,eur_foil,is_foil,is_etched,is_glossy,frame_effects_usd,border_color_usd,flavor_text_usd,frame_usd,full_art_usd,highres_images_usd,lang_usd,layout_usd,promo_types_usd,security_stamp_usd,watermark_usd
12661,2d911053-a026-4b20-ba2d-dbcc367c1413,Ajani Goldmane,en,2010-07-16,normal,True,highres_scan,{2}{W}{W},4.0,Legendary Planeswalker — Ajani,+1: You gain 2 life.\n−1: Put a +1/+1 counter ...,0,0,[W],[W],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,True,False,485d2468-18c8-42a4-9482-ca1c51e0470e,m11,Magic 2011,core,1,False,mythic,no_flavor_text,Aleksi Briclot,black,2003,False,False,True,False,6746.0,1920.0,"{'usd': '8.02', 'usd_foil': '15.11', 'usd_etch...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,4,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,8.02,15.11,3.50,9.48,True,False,False,NaN,NaN,NaN,8.02,NaN,8.02,NaN,NaN,NaN,NaN,NaN
61300,dcb2e0d6-3181-4d3d-a3b4-3896288b2e0e,Champion of Lambholt,en,2020-08-07,normal,True,highres_scan,{1}{G}{G},3.0,Creature — Human Warrior,Creatures with power less than Champion of Lam...,1,1,[G],[G],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,True,False,372dafe8-b5d1-4b81-998f-3ae96b59498a,2xm,Double Masters,masters,156,False,rare,no_flavor_text,Christopher Moeller,black,2015,False,False,True,False,446.0,5238.0,"{'usd': '0.36', 'usd_foil': '3.22', 'usd_etche...",no_promo,oval,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.36,3.22,0.34,1.65,True,False,False,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,NaN,0.36,NaN
48604,af3aa149-00fa-4932-b004-f64c8b5d3ca7,Gargoyle,en,2009-07-17,token,True,highres_scan,,0.0,Token Artifact Creature — Gargoyle,Flying,3,4,[],[],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],False,False,False,False,6d9f49c1-4784-4fd6-a9f1-05e36e25befe,tm10,Magic 2010 Tokens,token,8,False,common,no_flavor_text,Paul Bonner,black,2003,False,False,True,False,0.0,0.0,"{'usd': '0.25', 'usd_foil': None, 'usd_etched'...",[setpromo],no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.25,0,0,0,False,False,False,NaN,NaN,NaN,0.25,NaN,0.25,NaN,0.25,0.25,NaN,NaN


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53199 entries, 12661 to 1677
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  53199 non-null  object        
 1   name                53199 non-null  object        
 2   lang                53199 non-null  object        
 3   released_at         53199 non-null  datetime64[ns]
 4   layout              53199 non-null  object        
 5   highres_image       53199 non-null  bool          
 6   image_status        53199 non-null  object        
 7   mana_cost           53199 non-null  object        
 8   cmc                 53194 non-null  float64       
 9   type_line           53194 non-null  object        
 10  oracle_text         53199 non-null  object        
 11  power               53199 non-null  object        
 12  toughness           53199 non-null  object        
 13  colors              53199 non-null  object 

In [11]:
avg_frame_effects = train.frame_effects_usd.mean(skipna=True)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [9]:
avg_frame_effects = train.frame_effects_usd.mean(skipna=True)
avg_border_color = train.border_color_usd.mean()
avg_flavor_text = train.flavor_text_usd.mean()
avg_frame = train.frame_usd.mean()
avg_full_art = train.full_art_usd.mean()
avg_highres_images = train.highres_images_usd.mean()
avg_lang = train.lang_usd.mean()
avg_layout = train.layout_usd.mean()
avg_promo_types = train.promo_types_usd.mean()
avg_security_stamp = train.security_stamp_usd.mean()
avg_watermark = train.watermark_usd.mean()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [8]:
train.frame_effects_usd.value_counts()

0        1099
0.19       75
0.23       74
0.09       72
0.11       69
         ... 
3.37        1
15.05       1
5.89        1
25.67       1
21.98       1
Name: frame_effects_usd, Length: 864, dtype: int64